In [1]:
from app.dataManager import *
import math
import matplotlib.pyplot as plt

ana_new = AnaMaster(mask_bins=False, scale_to_HK=True)

Loading Density profile from: ../data/PREM.dat
MASK BINS:  False


In [2]:
plt.rcParams['text.usetex'] = True  # Enable LaTeX rendering
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 12 

In [3]:
def generate_neutrino_samples_latex_table():
    """Generate a LaTeX table for neutrino samples with their types."""
    
    # Mapping from reaction codes to neutrino types
    reaction_to_neutrino = {
        0: r'$\bar{\nu}_\mu$',
        3: r'$\nu_e$', 
        4: r'$\nu_\mu$',
        5: r'$\bar{\nu}_e$'
    }
    
    # Define samples in the requested order with their reaction maps
    samples_data = [
        # Sub-GeV samples
        ('Sub-GeV #nu_{e}-like', 3),
        ('Sub-GeV #bar{#nu}_{e}-like 0 n', 5),
        ('Sub-GeV #bar{#nu}_{e}-like 1 n', 5),
        ('Sub-GeV #nu_{#mu}-like', 4),
        ('Sub-GeV #bar{#nu}_{#mu}-like', 0),
        
        # Multi-GeV samples
        ('Multi-GeV #nu_{e}-like', 3),
        ('Multi-GeV #bar{#nu}_{e}-like 0 n', 5),
        ('Multi-GeV #bar{#nu}_{e}-like 1 n', 5),
        ('Multi-GeV #nu_{#mu}-like', 4),
        ('Multi-GeV #bar{#nu}_{#mu}-like', 0),
        
        # Multi-GeV Multi-Ring samples
        ('Multi-GeV Multi-Ring #nu_{e}-like', 3),
        ('Multi-GeV Multi-Ring #bar{#nu}_{e}-like', 5),
        ('Multi-GeV Multi-Ring #mu-like', 4),
        #('Multi-GeV Multi-Ring  Other', 4),
        
        # PC samples (assuming they correspond to muon neutrinos based on context)
        ('PC  Stopping', 4),  # Assuming muon neutrino
        ('PC  Through-going', 4),  # Assuming muon neutrino
        
        # Up-μ samples
        ('Up-#mu Stopping', 4),
        ('Up-#mu Non-Showering', 4),
        ('Up-#mu Showering', 4)
    ]
    
    # Generate LaTeX table using list approach
    latex_table = []
    
    # Table header
    latex_table.append(r"\begin{table}[htbp]")
    latex_table.append(r"\centering")
    latex_table.append(r"\caption{Neutrino Sample Classification}")
    latex_table.append(r"\label{tab:neutrino_samples}")
    latex_table.append(r"\begin{tabular}{c|l|c}")
    latex_table.append(r"\toprule")
    latex_table.append(r"Sample ID & Sample Name & Neutrino Type \\")
    latex_table.append(r"\midrule")
    
    # Add data rows
    for i, (sample_name, reaction_code) in enumerate(samples_data):
        # Convert sample name to LaTeX format
        latex_sample_name = sample_name
        
        # Replace neutrino symbols with proper LaTeX
        latex_sample_name = latex_sample_name.replace('#nu_{e}', r'$\nu_e$')
        latex_sample_name = latex_sample_name.replace('#bar{#nu}_{e}', r'$\bar{\nu}_e$')
        latex_sample_name = latex_sample_name.replace('#nu_{#mu}', r'$\nu_\mu$')
        latex_sample_name = latex_sample_name.replace('#bar{#nu}_{#mu}', r'$\bar{\nu}_\mu$')
        latex_sample_name = latex_sample_name.replace('#mu', r'$\mu$')
        
        # Get neutrino type
        neutrino_type = reaction_to_neutrino[reaction_code]
        
        # Add table row
        latex_table.append(f"{i+1:2d} & {latex_sample_name} & {neutrino_type} \\\\")
    
    # Table footer
    latex_table.append(r"\bottomrule")
    latex_table.append(r"\end{tabular}")
    latex_table.append(r"\end{table}")
    
    return '\n'.join(latex_table)

# Generate and print the LaTeX table
latex_table = generate_neutrino_samples_latex_table()
print(latex_table)

\begin{table}[htbp]
\centering
\caption{Neutrino Sample Classification}
\label{tab:neutrino_samples}
\begin{tabular}{c|l|c}
\toprule
Sample ID & Sample Name & Neutrino Type \\
\midrule
 1 & Sub-GeV $\nu_e$-like & $\nu_e$ \\
 2 & Sub-GeV $\bar{\nu}_e$-like 0 n & $\bar{\nu}_e$ \\
 3 & Sub-GeV $\bar{\nu}_e$-like 1 n & $\bar{\nu}_e$ \\
 4 & Sub-GeV $\nu_\mu$-like & $\nu_\mu$ \\
 5 & Sub-GeV $\bar{\nu}_\mu$-like & $\bar{\nu}_\mu$ \\
 6 & Multi-GeV $\nu_e$-like & $\nu_e$ \\
 7 & Multi-GeV $\bar{\nu}_e$-like 0 n & $\bar{\nu}_e$ \\
 8 & Multi-GeV $\bar{\nu}_e$-like 1 n & $\bar{\nu}_e$ \\
 9 & Multi-GeV $\nu_\mu$-like & $\nu_\mu$ \\
10 & Multi-GeV $\bar{\nu}_\mu$-like & $\bar{\nu}_\mu$ \\
11 & Multi-GeV Multi-Ring $\nu_e$-like & $\nu_e$ \\
12 & Multi-GeV Multi-Ring $\bar{\nu}_e$-like & $\bar{\nu}_e$ \\
13 & Multi-GeV Multi-Ring $\mu$-like & $\nu_\mu$ \\
14 & PC  Stopping & $\nu_\mu$ \\
15 & PC  Through-going & $\nu_\mu$ \\
16 & Up-$\mu$ Stopping & $\nu_\mu$ \\
17 & Up-$\mu$ Non-Showering & $\nu

In [4]:
from collections import defaultdict
from scipy.optimize import curve_fit
import numpy as np
import matplotlib.pyplot as plt
import os

def power_law(x, a, b):
    """Power law function: a * x^(-b)"""
    return a * np.power(x, -b)

def combine_and_fit(ana_master, sample_list, reaction_map, output_name=""):
    """
    Combine all bins and fit power laws for energy and angular resolution.
    
    Parameters:
    -----------
    ana_master : AnaMaster instance
        Initialized AnaMaster object
    sample_list : list
        List of sample names to process
    reaction_map : dict
        Dictionary mapping sample names to reaction indices
    output_name : str
        Name for output files
        
    Returns:
    --------
    dict
        Fit parameters: {'energy': {'a': float, 'b': float}, 
        'angular': {'a': float, 'b': float}}
    """
    
    # Collect all data points across all samples
    all_energies = []
    all_energy_resolutions = []
    all_angular_resolutions = []
    
    print(f"Collecting data from {len(sample_list)} samples...")
    
    # Process each sample
    for sample in ana_master.samples:
        sample_name = sample.name
        
        # Check if this is one of our target samples
        if sample_name in sample_list:
            # Skip if sample name contains '*'
            if '*' in sample_name:
                continue

            print(f"  Processing: {sample_name}")
            reaction_idx = reaction_map.get(sample_name, 0)
            
            # Process each bin in this sample
            for bin_idx in sample.bin_indices:
                # Get bin from the specified reaction
                ana_bin = ana_master.reactions[reaction_idx].ana_bins[bin_idx]
                
                if not ana_bin.ignore_bin and ana_bin.energy_avg > 0:
                    # Calculate metrics
                    energy_resolution = ana_bin.energy_rms / ana_bin.energy_avg
                    angular_resolution = ana_bin.cos_z_rms
                    
                    # Filter out points with ERMS/EAVE > 4 for Multi Ring analysis
                    if 'MultiRing' in output_name and energy_resolution > 4:
                        continue
                    
                    # Store all data points
                    all_energies.append(ana_bin.energy_avg)
                    all_energy_resolutions.append(energy_resolution)
                    all_angular_resolutions.append(angular_resolution)
    
    # Convert to numpy arrays
    all_energies = np.array(all_energies)
    all_energy_resolutions = np.array(all_energy_resolutions)
    all_angular_resolutions = np.array(all_angular_resolutions)
    
    print(f"Total data points collected: {len(all_energies)}")
    
    if len(all_energies) == 0:
        print("No data points found!")
        return {}
    
    # Store energy range info
    energy_range = {
        'min': np.min(all_energies),
        'max': np.max(all_energies),
        'n_points': len(all_energies)
    }
    
    # Fit power laws
    fit_results = {'energy_range': energy_range}
    
    try:
        # Fit energy resolution: RMS(E)/Avg(E) = a * E^(-b)
        e_popt, e_pcov = curve_fit(power_law, all_energies, all_energy_resolutions)
        fit_results['energy'] = {
            'a': e_popt[0],
            'b': e_popt[1],
            'a_err': np.sqrt(e_pcov[0,0]),
            'b_err': np.sqrt(e_pcov[1,1])
        }
        print(f"Energy resolution fit: a = {e_popt[0]:.4f} ± {np.sqrt(e_pcov[0,0]):.4f}, b = {e_popt[1]:.4f} ± {np.sqrt(e_pcov[1,1]):.4f}")
        
    except Exception as e:
        print(f"Energy resolution fit failed: {e}")
        fit_results['energy'] = None
    
    try:
        # Fit angular resolution: RMS(cos_theta) = a * E^(-b)
        a_popt, a_pcov = curve_fit(power_law, all_energies, all_angular_resolutions)
        fit_results['angular'] = {
            'a': a_popt[0],
            'b': a_popt[1],
            'a_err': np.sqrt(a_pcov[0,0]),
            'b_err': np.sqrt(a_pcov[1,1])
        }
        print(f"Angular resolution fit: a = {a_popt[0]:.4f} ± {np.sqrt(a_pcov[0,0]):.4f}, b = {a_popt[1]:.4f} ± {np.sqrt(a_pcov[1,1]):.4f}")
        
    except Exception as e:
        print(f"Angular resolution fit failed: {e}")
        fit_results['angular'] = None
    
    # Create visualization
    create_combined_fit_plot(all_energies, all_energy_resolutions, all_angular_resolutions, 
                          fit_results, output_name)
    
    return fit_results

def create_combined_fit_plot(energies, energy_resolutions, angular_resolutions, fit_results, output_name):
    """
    Create a 2-panel plot showing the combined fits.
    """
    
    # Create output directory
    output_dir = create_output_dir()
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 3))
    
    # Determine title based on output_name
    title_map = {
        'SubGeV': 'Sub-GeV',
        'MultiGeV': 'Multi-GeV', 
        'MultiRing': 'Multi-GeV Multi Ring',
        'UpMu': 'Up-$\\mu$'
    }
    
    plot_title = title_map.get(output_name, output_name.replace('_Simple', ''))
    
    # Add centered title above both subplots, closer to the plots
    fig.suptitle(plot_title, fontsize=14, y=0.90)
    
    # Energy resolution plot
    ax1.scatter(energies, energy_resolutions, alpha=0.6, color='blue', s=20, label='Data')
    
    if fit_results.get('energy') is not None:
        e_params = fit_results['energy']
        fit_x = np.logspace(np.log10(min(energies)), np.log10(max(energies)), 100)
        fit_y = power_law(fit_x, e_params['a'], e_params['b'])
        ax1.plot(fit_x, fit_y, 'r-', linewidth=2, 
                label=f"$a \\times E^{{-b}}$\n$a = {e_params['a']:.4f} \\pm {e_params['a_err']:.4f}$\n$b = {e_params['b']:.4f} \\pm {e_params['b_err']:.4f}$")
    
    ax1.set_xlabel('Energy (GeV)')
    ax1.set_ylabel(r'$E^{\mathrm{RMS}}/E^{\mathrm{AVG}}$')
    
    # Apply specific y-limits based on output_name
    if 'SubGeV' in output_name:
        ax1.set_ylim(0, 0.8)
    elif 'MultiRing' in output_name:
        ax1.set_ylim(0, 5)
    elif 'Up' in output_name:
        ax1.set_ylim(0, 8)
    else:
        ax1.set_ylim(0)
    
    ax1.grid(True, alpha=0.3)
    ax1.legend()
    
    # Angular resolution plot
    ax2.scatter(energies, angular_resolutions, alpha=0.6, color='green', s=20, label='Data')
    
    if fit_results.get('angular') is not None:
        a_params = fit_results['angular']
        fit_x = np.logspace(np.log10(min(energies)), np.log10(max(energies)), 100)
        fit_y = power_law(fit_x, a_params['a'], a_params['b'])
        ax2.plot(fit_x, fit_y, 'r-', linewidth=2,
                label=f"$a \\times E^{{-b}}$\n$a = {a_params['a']:.4f} \\pm {a_params['a_err']:.4f}$\n$b = {a_params['b']:.4f} \\pm {a_params['b_err']:.4f}$")
    
    ax2.set_xlabel('Energy (GeV)')
    ax2.set_ylabel(r'$\cos^{\mathrm{RMS}}_{\theta_z}$')

    # Apply specific y-limits based on output_name
    if 'MultiGeV' in output_name and 'MultiRing' not in output_name:
        ax2.set_ylim(0, 0.25)
    elif 'MultiRing' in output_name:
        ax2.set_ylim(0, 0.4)
    elif 'Up' in output_name:
        ax2.set_ylim(0, 0.2)
    elif 'Sub' in output_name:
        ax2.set_ylim(0, 1)
    else:
        ax2.set_ylim(0)
        
    ax2.grid(True, alpha=0.3)
    ax2.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'{output_name}_combined_fits.pdf'), 
                bbox_inches='tight', dpi=100)
    plt.close()
    
    print(f"Plot saved: {output_name}_combined_fits.pdf")

def create_output_dir(dir_name='output_SK_performance'):
    """Create a directory for output figures if it doesn't exist."""
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    return dir_name

def run_combined_subgev_analysis(ana_master):
    """Run the Sub-GeV analysis."""
    sub_gev_samples = [
        'Sub-GeV #nu_{e}-like',
        'Sub-GeV #nu_{#mu}-like',
        'Sub-GeV #bar{#nu}_{e}-like 0 n',
        'Sub-GeV #bar{#nu}_{e}-like 1 n',
        'Sub-GeV #bar{#nu}_{#mu}-like'
    ]
    
    sub_gev_reaction_map = {
        'Sub-GeV #nu_{e}-like': 3,
        'Sub-GeV #nu_{#mu}-like': 4,
        'Sub-GeV #bar{#nu}_{e}-like 0 n': 5,
        'Sub-GeV #bar{#nu}_{e}-like 1 n': 5,
        'Sub-GeV #bar{#nu}_{#mu}-like': 0
    }
    
    print("Running Sub-GeV combined analysis...")
    return combine_and_fit(ana_master, sub_gev_samples, sub_gev_reaction_map, "SubGeV")

def run_combined_multigev_analysis(ana_master):
    """Run the Multi-GeV analysis."""
    multi_gev_samples = [
        'Multi-GeV #nu_{e}-like',
        'Multi-GeV #bar{#nu}_{e}-like 0 n',
        'Multi-GeV #bar{#nu}_{e}-like 1 n',
        'Multi-GeV #nu_{#mu}-like',
        'Multi-GeV #bar{#nu}_{#mu}-like'
    ]
    
    multi_gev_reaction_map = {
        'Multi-GeV #nu_{e}-like': 3,
        'Multi-GeV #bar{#nu}_{e}-like 0 n': 5,
        'Multi-GeV #bar{#nu}_{e}-like 1 n': 5,
        'Multi-GeV #nu_{#mu}-like': 4,
        'Multi-GeV #bar{#nu}_{#mu}-like': 0
    }
    
    print("Running the Multi-GeV combined analysis...")
    return combine_and_fit(ana_master, multi_gev_samples, multi_gev_reaction_map, "MultiGeV")

def run_combined_multiring_analysis(ana_master):
    """Run the Multi-Ring analysis."""
    multi_ring_samples = [
        'Multi-GeV Multi-Ring #nu_{e}-like',
        'Multi-GeV Multi-Ring #bar{#nu}_{e}-like',
        'Multi-GeV Multi-Ring #mu-like',
        #'Multi-GeV Multi-Ring  Other'
    ]
    
    multi_ring_reaction_map = {
        'Multi-GeV Multi-Ring #nu_{e}-like': 3,
        'Multi-GeV Multi-Ring #bar{#nu}_{e}-like': 5,
        'Multi-GeV Multi-Ring #mu-like': 4,
        #'Multi-GeV Multi-Ring  Other': 4
    }
    
    print("Running the Multi-Ring combined analysis...")
    return combine_and_fit(ana_master, multi_ring_samples, multi_ring_reaction_map, "MultiRing")

def run_combined_upmu_analysis(ana_master):
    """Run the combined Up-μ analysis for Showering and Non-Showering samples only."""
    upmu_samples = [
        'Up-#mu Stopping',
        'Up-#mu Non-Showering',
        'Up-#mu Showering'
    ]
    
    upmu_reaction_map = {
        'Up-#mu Stopping': 4,
        'Up-#mu Non-Showering': 4,
        'Up-#mu Showering': 4
    }
    
    print("Running the Up-μ (Showering + Non-Showering) combined analysis...")
    return combine_and_fit(ana_master, upmu_samples, upmu_reaction_map, "UpMu")

def create_combined_summary_table(all_combined_results):
    """
    Generate LaTeX table summarizing all combined parametrizations.
    
    Parameters:
    -----------
    all_combined_results : dict
        Dictionary containing results from all analyses
        Format: {'SubGeV': results, 'MultiGeV': results, ...}
        
    Returns:
    --------
    str
        LaTeX formatted table
    """
    latex_table = []
    
    # Table header
    latex_table.append(r"\begin{table*}[htbp]")
    latex_table.append(r"\centering")
    latex_table.append(r"\small")
    latex_table.append(r"\caption{Parametrization for aggregated samples: $E^{\small{\textup{RMS}}}/E^{\small{\textup{AVG}}} = a_E \times E^{-b_E}$; $c^{\small{\textup{RMS}}}_{\theta_z} = a_A \times E^{-b_A}$}")
    latex_table.append(r"\label{tab:simple_combined_resolution}")
    latex_table.append(r"\hspace*{\fill}")
    latex_table.append(r"\begin{tabular}{l|@{\hspace{3pt}}c@{\hspace{3pt}}|@{\hspace{3pt}}c@{\hspace{3pt}}|@{\hspace{3pt}}c@{\hspace{3pt}}|@{\hspace{3pt}}c@{\hspace{3pt}}|c}")
    latex_table.append(r"\toprule")
    latex_table.append(r"& \multicolumn{2}{@{\hspace{3pt}}c@{\hspace{3pt}}|@{\hspace{3pt}}}{$E^{\small{\textup{RMS}}}/E^{\small{\textup{AVG}}}$} & \multicolumn{2}{@{\hspace{3pt}}c@{\hspace{3pt}}|}{$c^{\small{\textup{RMS}}}_{\theta_z}$} & \\")
    latex_table.append(r"Sample Group & $a_E$ & $b_E$ & $a_A$ & $b_A$ & $E_{\text{Range}}$ [GeV] \\")
    latex_table.append(r"\midrule")
    
    # Define the order and display names for sample groups
    group_order = [
        ('SubGeV', 'Sub-GeV'),
        ('MultiGeV', 'Multi-GeV'),
        ('MultiRing', 'Multi-Ring'),
        ('UpMu', 'Up-$\\mu$')
    ]
    
    # Process each group
    for group_key, group_display_name in group_order:
        if group_key not in all_combined_results:
            continue
            
        results = all_combined_results[group_key]
        
        if not results:  # Skip empty results
            continue
        
        # Extract parameters with error handling
        try:
            # Energy resolution parameters
            if results.get('energy') is not None:
                e_a = results['energy']['a']
                e_b = results['energy']['b']
                e_a_err = results['energy']['a_err']
                e_b_err = results['energy']['b_err']
                energy_params = f"{e_a:.2f} $\\pm$ {e_a_err:.2f} & {e_b:.2f} $\\pm$ {e_b_err:.2f}"
            else:
                energy_params = "-- & --"
            
            # Angular resolution parameters
            if results.get('angular') is not None:
                a_a = results['angular']['a']
                a_b = results['angular']['b']
                a_a_err = results['angular']['a_err']
                a_b_err = results['angular']['b_err']
                angular_params = f"{a_a:.2f} $\\pm$ {a_a_err:.2f} & {a_b:.2f} $\\pm$ {a_b_err:.2f}"
            else:
                angular_params = "-- & --"
            
            # Energy range and data points
            if 'energy_range' in results:
                e_range = results['energy_range']
                min_energy_mev = e_range['min']  # Keep in MeV for debugging
                max_energy_mev = e_range['max']  # Keep in MeV for debugging
                min_energy = min_energy_mev
                max_energy = max_energy_mev
                n_points = e_range['n_points']
                
                # Debug print
                print(f"  {group_key}: Energy range {min_energy_mev:.1f} - {max_energy_mev:.1f} MeV = {min_energy:.3f} - {max_energy:.3f} GeV")
                
                # Format energy ranges appropriately
                if min_energy < 0.01:
                    min_str = f"{min_energy:.3f}"
                elif min_energy < 0.1:
                    min_str = f"{min_energy:.2f}"
                else:
                    min_str = f"{min_energy:.1f}"
                    
                if max_energy < 1:
                    max_str = f"{max_energy:.2f}"
                elif max_energy < 10:
                    max_str = f"{max_energy:.1f}"
                else:
                    max_str = f"{max_energy:.0f}"
                    
                range_info = f"[{min_str}, {max_str}]"
            else:
                range_info = "-- & --"
            
            # Construct table row
            row = f"{group_display_name} & {energy_params} & {angular_params} & {range_info} \\\\"
            latex_table.append(row)
            
        except Exception as e:
            print(f"Error processing {group_key}: {e}")
            # Add placeholder row for failed processing
            row = f"{group_display_name} & -- & -- & -- & -- & -- & -- \\\\"
            latex_table.append(row)
    
    # Table footer
    latex_table.append(r"\bottomrule")
    latex_table.append(r"\end{tabular}")
    latex_table.append(r"\begin{tablenotes}")
    latex_table.append(r"\small")
    latex_table.append(r"\item \textbf{Model:} Energy resolution: RMS/avg$(E) = a_E \times E^{-b_E}$; Angular resolution: RMS$(\cos\theta_z) = a_A \times E^{-b_A}$")
    latex_table.append(r"\item \textbf{Method:} All bins from each sample group combined and fitted with single power law")
    latex_table.append(r"\end{tablenotes}")
    latex_table.append(r"\end{table}")
    
    return '\n'.join(latex_table)

def run_all_analyses_and_create_table(ana_master):
    """
    Run all combined analyses and create summary table.
    
    Parameters:
    -----------
    ana_master : AnaMaster instance
        Initialized AnaMaster object
        
    Returns:
    --------
    tuple
        (all_results_dict, latex_table_string)
    """
    
    print("="*60)
    print("RUNNING ALL COMBINED PARAMETRIZATION ANALYSES")
    print("="*60)
    
    # Run all analyses
    all_results = {}
    
    try:
        all_results['SubGeV'] = run_combined_subgev_analysis(ana_master)
    except Exception as e:
        print(f"Sub-GeV analysis failed: {e}")
        all_results['SubGeV'] = {}
    
    try:
        all_results['MultiGeV'] = run_combined_multigev_analysis(ana_master)
    except Exception as e:
        print(f"Multi-GeV analysis failed: {e}")
        all_results['MultiGeV'] = {}
    
    try:
        all_results['MultiRing'] = run_combined_multiring_analysis(ana_master)
    except Exception as e:
        print(f"Multi-Ring analysis failed: {e}")
        all_results['MultiRing'] = {}
    
    try:
        all_results['UpMu'] = run_combined_upmu_analysis(ana_master)
    except Exception as e:
        print(f"Up-μ analysis failed: {e}")
        all_results['UpMu'] = {}
    
    print("\n" + "="*60)
    print("GENERATING SUMMARY TABLE")
    print("="*60)
    
    # Create summary table
    latex_table = create_combined_summary_table(all_results)
    
    # Save table to file
    output_dir = create_output_dir()
    table_filename = os.path.join(output_dir, 'combined_parametrization_summary.tex')
    
    with open(table_filename, 'w') as f:
        f.write(latex_table)
    
    print(f"Summary table saved to: {table_filename}")
    print("\nLaTeX Table:")
    print("-" * 40)
    print(latex_table)
    
    return all_results, latex_table

In [11]:
# Run everything and get both results and table
all_results, latex_table = run_all_analyses_and_create_table(ana_new)

# # Or run individually:
# results = {
#     'SubGeV': run_combined_subgev_analysis(ana_new),
#     'MultiGeV': run_combined_multigev_analysis(ana_new),
#     'MultiRing': run_combined_multiring_analysis(ana_new),
#     'UpMu': run_combined_upmu_analysis(ana_new)
# }
# table = create_combined_summary_table(results)

RUNNING ALL COMBINED PARAMETRIZATION ANALYSES
Running Sub-GeV combined analysis...
  Processing: Sub-GeV #nu_{e}-like
  Processing: Sub-GeV #bar{#nu}_{e}-like 0 n
  Processing: Sub-GeV #bar{#nu}_{e}-like 1 n
  Processing: Sub-GeV #nu_{#mu}-like
  Processing: Sub-GeV #bar{#nu}_{#mu}-like
Total data points collected: 250
Energy resolution fit: a = 0.2645 ± 0.0031, b = 0.4261 ± 0.0144
Angular resolution fit: a = 0.3397 ± 0.0061, b = 0.3713 ± 0.0231
Plot saved: SubGeV_combined_fits.pdf
Running the Multi-GeV combined analysis...
  Processing: Multi-GeV #nu_{e}-like
  Processing: Multi-GeV #bar{#nu}_{e}-like 0 n
  Processing: Multi-GeV #bar{#nu}_{e}-like 1 n
  Processing: Multi-GeV #nu_{#mu}-like
  Processing: Multi-GeV #bar{#nu}_{#mu}-like
Total data points collected: 160
Energy resolution fit: a = 0.0534 ± 0.0080, b = -0.9215 ± 0.0452
Angular resolution fit: a = 0.2441 ± 0.0112, b = 0.4438 ± 0.0339
Plot saved: MultiGeV_combined_fits.pdf
Running the Multi-Ring combined analysis...
  Process

In [6]:
def fit_and_parametrize_samples(ana_master, sample_group_name, sample_list, reaction_map, colors_map=None):
    """
    Parameters:
    -----------
    ana_master : AnaMaster instance
        Initialized AnaMaster object
    sample_group_name : str
        Name of the sample group (for file naming)
    sample_list : list
        List of sample names to process
    reaction_map : dict
        Dictionary mapping sample names to reaction indices
    colors_map : dict, optional
        Dictionary mapping sample names to colors
        
    Returns:
    --------
    dict
        Fit parameters for all samples (including energy ranges)
    """
    # Create output directory
    output_dir = create_output_dir()
    
    # If no color map provided, generate one
    colors_map = None
    if colors_map is None:
        base_colors = ['blue', 'green', 'red', 'orange', 'purple', 'brown', 'pink', 'gray', 'olive', 'cyan']
        colors_map = {}
        for i, sample_name in enumerate(sample_list):
            colors_map[sample_name] = base_colors[i % len(base_colors)]
    
    # Data structure for grouping and parametrization
    data_by_sample_and_angle = defaultdict(lambda: defaultdict(list))
    fit_params = defaultdict(lambda: defaultdict(dict))
    
    # Process each sample
    for sample in ana_master.samples:
        sample_name = sample.name
        
        # Check if this is one of our target samples
        if sample_name in sample_list:
            # Skip if sample name contains '*'
            if '*' in sample_name:
                continue

            print(f"Processing: {sample_name}")
            reaction_idx = reaction_map.get(sample_name, 0)  # Default to reaction 0 if not specified
            
            # Process each bin in this sample
            for bin_idx in sample.bin_indices:
                # Get bin from the specified reaction
                ana_bin = ana_master.reactions[reaction_idx].ana_bins[bin_idx]
                
                if not ana_bin.ignore_bin and ana_bin.energy_avg > 0:
                    # Calculate metrics
                    energy_resolution = ana_bin.energy_rms / ana_bin.energy_avg
                    angular_resolution = ana_bin.cos_z_rms  # Use cos_z_rms directly
                    
                    # Get bin information from bin_manager to access bin edges
                    bin_info = ana_master.bin_manager.bin_info_df.iloc[bin_idx]
                    
                    # Create a key for the zenith angle bin using its edges
                    cosz_bin_key = f"{bin_info['CosZ_lo']:.1f}_{bin_info['CosZ_hi']:.1f}"
                    
                    # Calculate center of zenith bin for parametrization
                    cosz_center = (bin_info['CosZ_lo'] + bin_info['CosZ_hi']) / 2
                    
                    # Store metrics
                    data_by_sample_and_angle[sample_name][cosz_bin_key].append(
                        (ana_bin.energy_avg, energy_resolution, angular_resolution, cosz_center)
                    )
    
    # If no data was collected, return empty dict
    if not data_by_sample_and_angle:
        print(f"No data collected for {sample_group_name} samples")
        return {}
        
    # Define subplot permutation for 2x3 layout - you can modify this array to rearrange plots
    # New layout: [0,0], [0,1], [0,2], [1,0], [1,1], [1,2]
    # Current labels: Energy Res, Angular Res, Energy a_E, Energy b_E, Angular a_A, Angular b_A
    
    # Option 1: Default layout for 2x3
    subplot_order = [(0,0), (1,0), (0,1), (1,1), (0,2), (1,2)]
    
    # Map logical plot indices to actual subplot positions
    plot_mapping = {
        0: subplot_order[0],  # Energy Resolution vs Energy
        1: subplot_order[1],  # Angular Resolution vs Energy  
        2: subplot_order[2],  # Energy a parameters vs cos(zenith)
        3: subplot_order[3],  # Energy b parameters vs cos(zenith)
        4: subplot_order[4],  # Angular a parameters vs cos(zenith)
        5: subplot_order[5]   # Angular b parameters vs cos(zenith)
    }
    
    # Create plots for visualization and fitting (2 rows, 3 cols)
    fig, axs = plt.subplots(2, 3, figsize=(9, 5))
    
    # For creating a single legend
    legend_handles = []
    legend_labels = []
    
    # Process each sample with enhanced fitting strategy
    for sample_name in sorted(data_by_sample_and_angle.keys()):
        sample_color = colors_map[sample_name]
        formatted_name = latex_format_sample_name(sample_name)
        
        print(f"\nAnalyzing {sample_name}:")
        print(f"  Number of zenith bins: {len(data_by_sample_and_angle[sample_name])}")
        
        # Collect all energy values for this sample to determine energy range
        all_sample_energies = []
        for z_bin_data in data_by_sample_and_angle[sample_name].values():
            for energy, _, _, _ in z_bin_data:
                all_sample_energies.append(energy)
        
        # Store energy range for this sample
        if all_sample_energies:
            fit_params[sample_name]['energy_range'] = {
                'min': min(all_sample_energies),
                'max': max(all_sample_energies)
            }
        
        # Setup for second-level parametrization
        energy_a_params = []
        energy_b_params = []
        angular_a_params = []
        angular_b_params = []
        cosz_centers = []
        
        # Count successful fits per zenith bin
        successful_zenith_bins = 0
        
        # Process each zenith angle bin for this sample
        for z_bin in sorted(data_by_sample_and_angle[sample_name].keys()):
            points = data_by_sample_and_angle[sample_name][z_bin]
            
            # Sort points by energy
            points.sort(key=lambda x: x[0])
            
            print(f"    Zenith bin {z_bin}: {len(points)} points")
            
            # Extract data for plotting and fitting
            if len(points) >= 1:  # Relaxed condition - try to fit with any data
                energies = np.array([p[0] for p in points])
                e_resolutions = np.array([p[1] for p in points])
                a_resolutions = np.array([p[2] for p in points])
                cosz_center = points[0][3]  # All points in this bin have same center
                
                # Generate fit_x for plotting (consistent range for all fit types)
                fit_x = np.linspace(min(energies), max(energies), 100)
                
                # Fit energy resolution with fallback strategy
                energy_fit_success = False
                if len(points) >= 3:
                    # Try power law fit first
                    try:
                        e_popt, e_pcov = curve_fit(power_law, energies, e_resolutions)
                        fit_y = power_law(fit_x, *e_popt)
                        
                        # Store parameters
                        fit_params[sample_name]['energy_res'][z_bin] = {
                            'a': e_popt[0],
                            'b': e_popt[1],
                            'cosz_center': cosz_center,
                            'fit_type': 'power_law'
                        }
                        
                        # Collect for second level parametrization
                        energy_a_params.append(e_popt[0])
                        energy_b_params.append(e_popt[1])
                        energy_fit_success = True
                        
                        # Plot original data and fit - USE PERMUTED POSITION
                        pos = plot_mapping[0]  # Energy Resolution plot
                        axs[pos].scatter(energies, e_resolutions, color=sample_color, alpha=0.6)
                        line = axs[pos].plot(fit_x, fit_y, '-', color=sample_color, alpha=0.7)[0]
                        
                        print(f"      Energy resolution: Power law fit successful (a={e_popt[0]:.4f}, b={e_popt[1]:.4f})")
                        
                    except Exception as e:
                        print(f"      Energy resolution: Power law fit failed: {e}")
                
                if not energy_fit_success and len(points) >= 2:
                    # Try simple linear fit: σ = α + β*E
                    try:
                        coeffs = np.polyfit(energies, e_resolutions, 1)
                        alpha_param = coeffs[1]  # intercept
                        beta_param = coeffs[0]   # slope
                        
                        fit_params[sample_name]['energy_res'][z_bin] = {
                            'a': alpha_param,
                            'b': beta_param,
                            'cosz_center': cosz_center,
                            'fit_type': 'linear'
                        }
                        
                        energy_a_params.append(alpha_param)
                        energy_b_params.append(beta_param)
                        energy_fit_success = True
                        
                        # Plot data and fit line - USE PERMUTED POSITION
                        fit_y = linear_function(fit_x, alpha_param, beta_param)
                        pos = plot_mapping[0]  # Energy Resolution plot
                        axs[pos].scatter(energies, e_resolutions, color=sample_color, alpha=0.6)
                        line = axs[pos].plot(fit_x, fit_y, '-', color=sample_color, alpha=0.7)[0]
                        
                        print(f"      Energy resolution: Linear fit (α={alpha_param:.4f}, β={beta_param:.6f})")
                        
                    except Exception as e:
                        print(f"      Energy resolution: Linear fit failed: {e}")
                
                if not energy_fit_success:
                    # Final fallback - use constant (average resolution)
                    avg_resolution = np.mean(e_resolutions)
                    fit_params[sample_name]['energy_res'][z_bin] = {
                        'a': avg_resolution,
                        'b': 0.0,
                        'cosz_center': cosz_center,
                        'fit_type': 'constant'
                    }
                    
                    energy_a_params.append(avg_resolution)
                    energy_b_params.append(0.0)
                    
                    # Plot data and constant line - USE PERMUTED POSITION
                    fit_y = np.full_like(fit_x, avg_resolution)
                    pos = plot_mapping[0]  # Energy Resolution plot
                    axs[pos].scatter(energies, e_resolutions, color=sample_color, alpha=0.6)
                    line = axs[pos].plot(fit_x, fit_y, ':', color=sample_color, alpha=0.7)[0]
                    
                    print(f"      Energy resolution: Constant fit (value={avg_resolution:.4f})")
                
                # Similar process for angular resolution
                angular_fit_success = False
                if len(points) >= 3:
                    try:
                        a_popt, a_pcov = curve_fit(power_law, energies, a_resolutions)
                        fit_y = power_law(fit_x, *a_popt)
                        
                        fit_params[sample_name]['angular_res'][z_bin] = {
                            'a': a_popt[0],
                            'b': a_popt[1],
                            'cosz_center': cosz_center,
                            'fit_type': 'power_law'
                        }
                        
                        angular_a_params.append(a_popt[0])
                        angular_b_params.append(a_popt[1])
                        angular_fit_success = True
                        
                        # USE PERMUTED POSITION
                        pos = plot_mapping[1]  # Angular Resolution plot
                        axs[pos].scatter(energies, a_resolutions, color=sample_color, alpha=0.6)
                        axs[pos].plot(fit_x, fit_y, '-', color=sample_color, alpha=0.7)
                        
                        print(f"      Angular resolution: Power law fit successful (a={a_popt[0]:.4f}, b={a_popt[1]:.4f})")
                        
                    except Exception as e:
                        print(f"      Angular resolution: Power law fit failed: {e}")
                
                if not angular_fit_success and len(points) >= 2:
                    # Try simple linear fit for angular resolution
                    try:
                        coeffs = np.polyfit(energies, a_resolutions, 1)
                        alpha_param = coeffs[1]  # intercept
                        beta_param = coeffs[0]   # slope
                        
                        fit_params[sample_name]['angular_res'][z_bin] = {
                            'a': alpha_param,
                            'b': beta_param,
                            'cosz_center': cosz_center,
                            'fit_type': 'linear'
                        }
                        
                        angular_a_params.append(alpha_param)
                        angular_b_params.append(beta_param)
                        angular_fit_success = True
                        
                        # Plot data and fit line - USE PERMUTED POSITION
                        fit_y = linear_function(fit_x, alpha_param, beta_param)
                        pos = plot_mapping[1]  # Angular Resolution plot
                        axs[pos].scatter(energies, a_resolutions, color=sample_color, alpha=0.6)
                        axs[pos].plot(fit_x, fit_y, '-', color=sample_color, alpha=0.7)
                        
                        print(f"      Angular resolution: Linear fit (α={alpha_param:.4f}, β={beta_param:.6f})")
                        
                    except Exception as e:
                        print(f"      Angular resolution: Linear fit failed: {e}")
                
                if not angular_fit_success:
                    avg_resolution = np.mean(a_resolutions)
                    fit_params[sample_name]['angular_res'][z_bin] = {
                        'a': avg_resolution,
                        'b': 0.0,
                        'cosz_center': cosz_center,
                        'fit_type': 'constant'
                    }
                    
                    angular_a_params.append(avg_resolution)
                    angular_b_params.append(0.0)
                    
                    # Plot data and constant line - USE PERMUTED POSITION
                    fit_y = np.full_like(fit_x, avg_resolution)
                    pos = plot_mapping[1]  # Angular Resolution plot
                    axs[pos].scatter(energies, a_resolutions, color=sample_color, alpha=0.6)
                    axs[pos].plot(fit_x, fit_y, ':', color=sample_color, alpha=0.7)
                    
                    print(f"      Angular resolution: Constant fit (value={avg_resolution:.4f})")
                
                # Add zenith center if not already present
                if cosz_center not in cosz_centers:
                    cosz_centers.append(cosz_center)
                
                successful_zenith_bins += 1
        
        # Second-level parametrization with flexible polynomial fitting
        if successful_zenith_bins > 0:
            cosz_centers = np.array(cosz_centers)
            
            print(f"  Second-level parametrization with {successful_zenith_bins} zenith bins")
            
            # Fit energy resolution parameters vs cos(zenith) with flexible order
            if energy_a_params:
                energy_a_params = np.array(energy_a_params)
                energy_b_params = np.array(energy_b_params)
                
                # Fit a parameters
                a_coeffs, a_order, a_fit_type = fit_polynomial_flexible(cosz_centers, energy_a_params)
                print(f"    Energy a params: {a_fit_type} fit (order {a_order})")
                
                # Fit b parameters  
                b_coeffs, b_order, b_fit_type = fit_polynomial_flexible(cosz_centers, energy_b_params)
                print(f"    Energy b params: {b_fit_type} fit (order {b_order})")
                
                # Store parametrization
                fit_params[sample_name]['energy_res_params'] = {
                    'a': a_coeffs,
                    'b': b_coeffs,
                    'a_fit_type': a_fit_type,
                    'b_fit_type': b_fit_type,
                    'a_order': a_order,
                    'b_order': b_order
                }
                
                # Plot if we have enough points to visualize
                if len(cosz_centers) > 1:
                    fit_cosz = np.linspace(min(cosz_centers), max(cosz_centers), 100)
                    fit_energy_a = np.polyval(a_coeffs, fit_cosz) if a_order > 0 else np.full_like(fit_cosz, a_coeffs[0])
                    fit_energy_b = np.polyval(b_coeffs, fit_cosz) if b_order > 0 else np.full_like(fit_cosz, b_coeffs[0])
                    
                    # USE PERMUTED POSITIONS
                    pos_a = plot_mapping[2]  # Energy a parameters
                    pos_b = plot_mapping[3]  # Energy b parameters
                    
                    axs[pos_a].scatter(cosz_centers, energy_a_params, color=sample_color, marker='o')
                    line = axs[pos_a].plot(fit_cosz, fit_energy_a, '-', color=sample_color)[0]
                    
                    axs[pos_b].scatter(cosz_centers, energy_b_params, color=sample_color, marker='s')
                    axs[pos_b].plot(fit_cosz, fit_energy_b, '-', color=sample_color)
                    
                    # Add to legend only once per sample
                    if sample_name not in [label for label in legend_labels]:
                        legend_handles.append(line)
                        legend_labels.append(formatted_name)
            
            # Similar for angular resolution parameters
            if angular_a_params:
                angular_a_params = np.array(angular_a_params)
                angular_b_params = np.array(angular_b_params)
                
                a_coeffs, a_order, a_fit_type = fit_polynomial_flexible(cosz_centers, angular_a_params)
                b_coeffs, b_order, b_fit_type = fit_polynomial_flexible(cosz_centers, angular_b_params)
                
                print(f"    Angular a params: {a_fit_type} fit (order {a_order})")
                print(f"    Angular b params: {b_fit_type} fit (order {b_order})")
                
                fit_params[sample_name]['angular_res_params'] = {
                    'a': a_coeffs,
                    'b': b_coeffs,
                    'a_fit_type': a_fit_type,
                    'b_fit_type': b_fit_type,
                    'a_order': a_order,
                    'b_order': b_order
                }
                
                # Plot angular resolution parameters - USE PERMUTED POSITIONS
                if len(cosz_centers) > 1:
                    fit_cosz = np.linspace(min(cosz_centers), max(cosz_centers), 100)
                    fit_angular_a = np.polyval(a_coeffs, fit_cosz) if a_order > 0 else np.full_like(fit_cosz, a_coeffs[0])
                    fit_angular_b = np.polyval(b_coeffs, fit_cosz) if b_order > 0 else np.full_like(fit_cosz, b_coeffs[0])
                    
                    pos_a = plot_mapping[4]  # Angular a parameters
                    pos_b = plot_mapping[5]  # Angular b parameters
                    
                    axs[pos_a].scatter(cosz_centers, angular_a_params, color=sample_color, marker='o')
                    axs[pos_a].plot(fit_cosz, fit_angular_a, '-', color=sample_color)
                    
                    axs[pos_b].scatter(cosz_centers, angular_b_params, color=sample_color, marker='s')
                    axs[pos_b].plot(fit_cosz, fit_angular_b, '-', color=sample_color)
        
        else:
            print(f"  No successful zenith bin fits - attempting energy-only parametrization")
            
            # Fallback to energy-only parametrization
            energy_only_params = fit_energy_only_parametrization(data_by_sample_and_angle[sample_name])
            
            if energy_only_params:
                fit_params[sample_name]['energy_only_params'] = energy_only_params
                print(f"    Energy-only parametrization successful")
                
                # Add a simple entry to legend
                legend_handles.append(plt.Line2D([0], [0], color=sample_color, linestyle='-'))
                legend_labels.append(f"{formatted_name} (E-only)")
            else:
                print(f"    Failed to parametrize {sample_name}")
    
    # Configure plots - APPLY LABELS TO PERMUTED POSITIONS
    if legend_handles:
        y_offset = 1.06
        #print('AAAAAAAAAAAAAAAAAAA: ', sample_group_name)
        if 'Other' in sample_group_name or 'Ring' in sample_group_name:
            y_offset = 1.02
        
        fig.legend(
            legend_handles, 
            legend_labels, 
            loc='upper center', 
            bbox_to_anchor=(0.5, y_offset), 
            ncol=min(3, len(legend_handles)), 
            frameon=True, 
            fancybox=True, 
            shadow=True
        )

    # Define labels for each logical plot
    plot_labels = {
        0: ('Energy (GeV)', r'$E^{\textup{RMS}}/E^{\textup{AVG}}$'),
        1: ('Energy (GeV)', r'$c^{\textup{RMS}}_{\theta_z}$'),
        2: (r'$\cos(\theta_z)$', r'$a_E$'),
        3: (r'$\cos(\theta_z)$', r'$b_E$'),
        4: (r'$\cos(\theta_z)$', r'$a_A$'),
        5: (r'$\cos(\theta_z)$', r'$b_A$')
    }
    
    # Apply labels to the permuted positions
    for logical_idx, (xlabel, ylabel) in plot_labels.items():
        pos = plot_mapping[logical_idx]
        axs[pos].set_xlabel(xlabel)
        axs[pos].set_ylabel(ylabel)
        
        # Set log scale for energy plots
        if logical_idx in [0, 1] and 'Sub-GeV' not in sample_group_name:
            axs[pos].set_xscale('log')
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])  # Make room for legend at top
    plt.savefig(os.path.join(output_dir, f'{sample_group_name}_resolution_parametrization.pdf'), 
                bbox_inches='tight', dpi=100)
    plt.close()
    
    # Print summary
    print(f"\n{sample_group_name} - Resolution Parametrization Summary:")
    total_samples = len(data_by_sample_and_angle)
    parametrized_samples = len(fit_params)
    
    print(f"  Total samples processed: {total_samples}")
    print(f"  Successfully parametrized: {parametrized_samples}")
    
    for sample_name, params in fit_params.items():
        if 'energy_range' in params:
            energy_range = params['energy_range']
            print(f"    {sample_name}: Energy range [{energy_range['min']:.1f}, {energy_range['max']:.1f}] MeV")
        
        if 'energy_res_params' in params:
            print(f"      → Full zenith-energy parametrization")
        elif 'energy_only_params' in params:
            print(f"      → Energy-only parametrization")
        else:
            print(f"      → Partial parametrization")
    
    return fit_params

def create_power_law_table(all_fit_params):
    """
    Generate LaTeX table for Power-law samples only.
    Modified to use ID instead of sample names and remove group column.
    """
    
    # Sample ID mapping (index + 1)
    samples_data = [
        # Sub-GeV samples
        ('Sub-GeV #nu_{e}-like', 3),
        ('Sub-GeV #bar{#nu}_{e}-like 0 n', 5),
        ('Sub-GeV #bar{#nu}_{e}-like 1 n', 5),
        ('Sub-GeV #nu_{#mu}-like', 4),
        ('Sub-GeV #bar{#nu}_{#mu}-like', 0),
        
        # Multi-GeV samples
        ('Multi-GeV #nu_{e}-like', 3),
        ('Multi-GeV #bar{#nu}_{e}-like 0 n', 5),
        ('Multi-GeV #bar{#nu}_{e}-like 1 n', 5),
        ('Multi-GeV #nu_{#mu}-like', 4),
        ('Multi-GeV #bar{#nu}_{#mu}-like', 0),
        
        # Multi-GeV Multi-Ring samples
        ('Multi-GeV Multi-Ring #nu_{e}-like', 3),
        ('Multi-GeV Multi-Ring #bar{#nu}_{e}-like', 5),
        ('Multi-GeV Multi-Ring #mu-like', 4),
        #('Multi-GeV Multi-Ring  Other', 4),
        
        # PC samples (assuming they correspond to muon neutrinos based on context)
        ('PC  Stopping', 4),  # Assuming muon neutrino
        ('PC  Through-going', 4),  # Assuming muon neutrino
        
        # Up-μ samples
        ('Up-#mu Stopping', 4),
        ('Up-#mu Non-Showering', 4),
        ('Up-#mu Showering', 4)
    ]
    
    # Create sample name to ID mapping
    sample_to_id = {}
    for i, (sample_name, _) in enumerate(samples_data):
        sample_to_id[sample_name] = i + 1  # ID starts from 1
    
    latex_table = []
    
    # Table header for power-law samples
    latex_table.append(r"\begin{table*}[htbp]")
    latex_table.append(r"\centering")
    latex_table.append(r"\small")
    latex_table.append(r"\caption{Power-Law Parametrization: $E^{\small{\textup{RMS}}}/E^{\small{\textup{AVG}}} = a_E \times E^{-b_E}$; $c^{\small{\textup{RMS}}}_{\theta_z} = a_A \times E^{-b_A}$ (coefficients $\times 10$)}")
    latex_table.append(r"\label{tab:power_law_performance}")
    latex_table.append(r"\hspace*{\fill}")
    latex_table.append(r"\begin{tabular}{l|@{\hspace{3pt}}r@{\hspace{3pt}}r@{\hspace{3pt}}r@{\hspace{3pt}}|@{\hspace{3pt}}r@{\hspace{3pt}}r@{\hspace{3pt}}r@{\hspace{3pt}}|@{\hspace{3pt}}r@{\hspace{3pt}}r@{\hspace{3pt}}r@{\hspace{3pt}}|@{\hspace{3pt}}r@{\hspace{3pt}}r@{\hspace{3pt}}r@{\hspace{3pt}}|c}")
    latex_table.append(r"\toprule")
    latex_table.append(r"& \multicolumn{6}{@{\hspace{3pt}}c@{\hspace{3pt}}|@{\hspace{3pt}}}{$E^{\small{\textup{RMS}}}/E^{\small{\textup{AVG}}}$} & \multicolumn{6}{@{\hspace{3pt}}c@{\hspace{3pt}}|}{$c^{\small{\textup{RMS}}}_{\theta_z}$} & \\")
    latex_table.append(r"\midrule")
    latex_table.append(r"& \multicolumn{3}{@{\hspace{3pt}}c@{\hspace{3pt}}|@{\hspace{3pt}}}{$a_E$} & \multicolumn{3}{@{\hspace{3pt}}c@{\hspace{3pt}}|@{\hspace{3pt}}}{$b_E$} & \multicolumn{3}{@{\hspace{3pt}}c@{\hspace{3pt}}|@{\hspace{3pt}}}{$a_A$} & \multicolumn{3}{@{\hspace{3pt}}c@{\hspace{3pt}}|}{$b_A$} & \\")
    latex_table.append(r"ID & $c_0$ & $c_1$ & $c_2$ & $c_0$ & $c_1$ & $c_2$ & $c_0$ & $c_1$ & $c_2$ & $c_0$ & $c_1$ & $c_2$ & $E_{Range}$ \\")
    latex_table.append(r"\midrule")
    
    power_law_samples_found = 0
    
    # Process each group for power-law samples only
    for group_name, fit_params in sorted(all_fit_params.items()):
        if not fit_params:
            continue
            
        for sample_name in sorted(fit_params.keys()):
            # Only include power-law samples
            param_type = classify_parametrization_type(fit_params[sample_name])
            if param_type != "Power-law":
                continue
                
            power_law_samples_found += 1
            print(f"Adding to power-law table: {sample_name} (classified as {param_type})")
            
            # Get sample ID
            sample_id = sample_to_id.get(sample_name, "?")  # Use "?" if sample not found
            
            # Extract parameters
            energy_a_params, energy_b_params, angular_a_params, angular_b_params = extract_parameters(fit_params[sample_name])
            
            # Get energy range
            energy_range = get_energy_range(fit_params[sample_name])
            
            # Add row to table
            row = f"{sample_id} & "
            
            # Add all 12 parameters + energy range
            row += f"{energy_a_params[0]} & {energy_a_params[1]} & {energy_a_params[2]} & "
            row += f"{energy_b_params[0]} & {energy_b_params[1]} & {energy_b_params[2]} & "
            row += f"{angular_a_params[0]} & {angular_a_params[1]} & {angular_a_params[2]} & "
            row += f"{angular_b_params[0]} & {angular_b_params[1]} & {angular_b_params[2]} & "
            row += f"{energy_range} \\\\"
            
            latex_table.append(row)
    
    print(f"\nTotal power-law samples added to table: {power_law_samples_found}")
    
    # Table footer
    latex_table.append(r"\bottomrule")
    latex_table.append(r"\end{tabular}")
    latex_table.append(r"\hspace*{\fill}")
    latex_table.append(r"\begin{tablenotes}")
    latex_table.append(r"\small")
    latex_table.append(r"\item \textbf{Zenith dependence:} $a_E = c_0 + c_1 \cos\theta_z + c_2 \cos^2\theta_z$ (analogously for $b_E$, $a_A$, $b_A$).")
    latex_table.append(r"\item \textbf{Range:} Energy range of bins used for fitting each sample in GeV.")
    latex_table.append(r"\end{tablenotes}")
    latex_table.append(r"\end{table*}")
    
    return '\n'.join(latex_table)



def extract_linear_parameters(sample_params):
    """Extract linear parameters for a sample: σ = α + β*E
    Multiplies all coefficients by 1000 to make numbers shorter."""
    energy_alpha_params = ["--", "--", "--"]
    energy_beta_params = ["--", "--", "--"]
    angular_alpha_params = ["--", "--", "--"]
    angular_beta_params = ["--", "--", "--"]
    
    if 'energy_only_params' in sample_params:
        # Energy-only parametrization
        e_params = sample_params['energy_only_params']['energy']
        a_params = sample_params['energy_only_params']['angular']
        
        energy_alpha_params = [f"{e_params['a']*10:.2f}", "0.00", "0.00"]
        energy_beta_params = [f"{e_params['b']*10:.2f}", "0.00", "0.00"]
        angular_alpha_params = [f"{a_params['a']*10:.2f}", "0.00", "0.00"]
        angular_beta_params = [f"{a_params['b']*10:.2f}", "0.00", "0.00"]
        
    elif 'energy_res_params' in sample_params and 'angular_res_params' in sample_params:
        # Full parametrization
        e_res = sample_params['energy_res_params']
        a_res = sample_params['angular_res_params']
        
        # Energy resolution coefficients (multiply by 1000)
        energy_alpha_coeffs = (e_res['a'] + [0, 0, 0])[:3]
        energy_beta_coeffs = (e_res['b'] + [0, 0, 0])[:3]
        energy_alpha_params = [f"{x*10:.2f}" for x in energy_alpha_coeffs]
        energy_beta_params = [f"{x*10:.2f}" for x in energy_beta_coeffs]
        
        # Angular resolution coefficients (multiply by 1000)
        angular_alpha_coeffs = (a_res['a'] + [0, 0, 0])[:3]
        angular_beta_coeffs = (a_res['b'] + [0, 0, 0])[:3]
        angular_alpha_params = [f"{x*10:.2f}" for x in angular_alpha_coeffs]
        angular_beta_params = [f"{x*10:.2f}" for x in angular_beta_coeffs]
    
    return energy_alpha_params, energy_beta_params, angular_alpha_params, angular_beta_params


def create_linear_table(all_fit_params):
    """
    Fixed version of create_linear_table using the corrected classification.
    Modified to use ID instead of sample names and remove group column.
    """
    
    # Sample ID mapping (index + 1)
    samples_data = [
        # Sub-GeV samples
        ('Sub-GeV #nu_{e}-like', 3),
        ('Sub-GeV #bar{#nu}_{e}-like 0 n', 5),
        ('Sub-GeV #bar{#nu}_{e}-like 1 n', 5),
        ('Sub-GeV #nu_{#mu}-like', 4),
        ('Sub-GeV #bar{#nu}_{#mu}-like', 0),
        
        # Multi-GeV samples
        ('Multi-GeV #nu_{e}-like', 3),
        ('Multi-GeV #bar{#nu}_{e}-like 0 n', 5),
        ('Multi-GeV #bar{#nu}_{e}-like 1 n', 5),
        ('Multi-GeV #nu_{#mu}-like', 4),
        ('Multi-GeV #bar{#nu}_{#mu}-like', 0),
        
        # Multi-GeV Multi-Ring samples
        ('Multi-GeV Multi-Ring #nu_{e}-like', 3),
        ('Multi-GeV Multi-Ring #bar{#nu}_{e}-like', 5),
        ('Multi-GeV Multi-Ring #mu-like', 4),
        #('Multi-GeV Multi-Ring  Other', 4),
        
        # PC samples (assuming they correspond to muon neutrinos based on context)
        ('PC  Stopping', 4),  # Assuming muon neutrino
        ('PC  Through-going', 4),  # Assuming muon neutrino
        
        # Up-μ samples
        ('Up-#mu Stopping', 4),
        ('Up-#mu Non-Showering', 4),
        ('Up-#mu Showering', 4)
    ]
    
    # Create sample name to ID mapping
    sample_to_id = {}
    for i, (sample_name, _) in enumerate(samples_data):
        sample_to_id[sample_name] = i + 1  # ID starts from 1
    
    latex_table = []

    # Table header for linear samples
    latex_table.append(r"\begin{table*}[htbp]")
    latex_table.append(r"\centering")
    latex_table.append(r"\small")
    latex_table.append(r"\caption{Linear Parametrization: $E^{\small{\textup{RMS}}}/E^{\small{\textup{AVG}}} = a_E + b_E \times E$; $c^{\small{\textup{RMS}}}_{\theta_z} = a_A + b_A \times E$ (coefficients $\times 10$)}")
    latex_table.append(r"\label{tab:linear_performance}")
    latex_table.append(r"\hspace*{\fill}") 
    latex_table.append(r"\begin{tabular}{l|@{\hspace{3pt}}r@{\hspace{3pt}}r@{\hspace{3pt}}r@{\hspace{3pt}}|@{\hspace{3pt}}r@{\hspace{3pt}}r@{\hspace{3pt}}r@{\hspace{3pt}}|@{\hspace{3pt}}r@{\hspace{3pt}}r@{\hspace{3pt}}r@{\hspace{3pt}}|@{\hspace{3pt}}r@{\hspace{3pt}}r@{\hspace{3pt}}r@{\hspace{3pt}}|c}")   
    latex_table.append(r"\toprule")
    latex_table.append(r"& \multicolumn{6}{@{\hspace{3pt}}c@{\hspace{3pt}}|@{\hspace{3pt}}}{$E^{\small{\textup{RMS}}}/E^{\small{\textup{AVG}}}$} & \multicolumn{6}{@{\hspace{3pt}}c@{\hspace{3pt}}|}{$c^{\small{\textup{RMS}}}_{\theta_z}$} & \\")
    latex_table.append(r"\midrule")
    latex_table.append(r"& \multicolumn{3}{@{\hspace{3pt}}c@{\hspace{3pt}}|@{\hspace{3pt}}}{$a_E$} & \multicolumn{3}{@{\hspace{3pt}}c@{\hspace{3pt}}|@{\hspace{3pt}}}{$b_E$} & \multicolumn{3}{@{\hspace{3pt}}c@{\hspace{3pt}}|@{\hspace{3pt}}}{$a_A$} & \multicolumn{3}{@{\hspace{3pt}}c@{\hspace{3pt}}|}{$b_A$} & \\")
    latex_table.append(r"ID & $c_0$ & $c_1$ & $c_2$ & $c_0$ & $c_1$ & $c_2$ & $c_0$ & $c_1$ & $c_2$ & $c_0$ & $c_1$ & $c_2$ & $E_{Range}$ \\")
    latex_table.append(r"\midrule")
    
    linear_samples_found = 0
    
    # Process each group for linear samples only
    for group_name, fit_params in sorted(all_fit_params.items()):
        if not fit_params:
            continue
            
        for sample_name in sorted(fit_params.keys()):
            # Use FIXED classification function
            param_type = classify_parametrization_type(fit_params[sample_name])
            
            # Only include linear samples
            if param_type != "Linear":
                continue
                
            linear_samples_found += 1
            print(f"Adding to linear table: {sample_name} (classified as {param_type})")
            
            # Get sample ID
            sample_id = sample_to_id.get(sample_name, "?")  # Use "?" if sample not found
                
            # Extract parameters for linear fits
            energy_alpha_params, energy_beta_params, angular_alpha_params, angular_beta_params = extract_linear_parameters(fit_params[sample_name])
            
            # Get energy range
            energy_range = get_energy_range(fit_params[sample_name])
            
            # Add row to table
            row = f"{sample_id} & "
            
            # Add all 12 parameters + energy range
            row += f"{energy_alpha_params[0]} & {energy_alpha_params[1]} & {energy_alpha_params[2]} & "
            row += f"{energy_beta_params[0]} & {energy_beta_params[1]} & {energy_beta_params[2]} & "
            row += f"{angular_alpha_params[0]} & {angular_alpha_params[1]} & {angular_alpha_params[2]} & "
            row += f"{angular_beta_params[0]} & {angular_beta_params[1]} & {angular_beta_params[2]} & "
            row += f"{energy_range} \\\\"
            
            latex_table.append(row)
    
    print(f"\nTotal linear samples added to table: {linear_samples_found}")
    
    # Table footer
    latex_table.append(r"\bottomrule")
    latex_table.append(r"\end{tabular}")
    latex_table.append(r"\hspace*{\fill}")
    latex_table.append(r"\begin{tablenotes}")
    latex_table.append(r"\small")
    latex_table.append(r"\item \textbf{Zenith dependence:} $a_E = c_0 + c_1 \cos\theta_z + c_2 \cos^2\theta_z$ (analogously for $b_E$, $a_A$, $b_A$).")
    latex_table.append(r"\item \textbf{Range:} Energy range of bins used for fitting each sample in GeV.")
    latex_table.append(r"\end{tablenotes}")
    latex_table.append(r"\end{table*}")
    
    return '\n'.join(latex_table)

def create_constant_table(all_fit_params):
    """
    Generate LaTeX table for Constant samples.
    
    Parameters:
    -----------
    all_fit_params : dict
        Dictionary containing all fit parameters
        
    Returns:
    --------
    str
        LaTeX formatted table for constant samples
    """
    latex_table = []
    
    # Table header for constant samples
    latex_table.append(r"\begin{table*}[htbp]")
    latex_table.append(r"\centering")
    latex_table.append(r"\scriptsize")
    latex_table.append(r"\caption{Constant Parametrization: RMS/avg$(E) = a_E; RMS $(\cos\theta_z) = a_A$}")
    latex_table.append(r"\label{tab:constant_resolution}")
    latex_table.append(r"\begin{tabular}{ll|rrr|rrr|c}")
    latex_table.append(r"\toprule")
    latex_table.append(r"& & \multicolumn{3}{c|}{RMS/avg$(E)$} & \multicolumn{3}{c|}{RMS $(\cos\theta_z)$} & \\")
    latex_table.append(r"& & \multicolumn{3}{c|}{$\sigma_E(\cos\theta_z)$} & \multicolumn{3}{c|}{$\sigma_A(\cos\theta_z)$} & \\")
    latex_table.append(r"Group & Sample & $k_0$ & $k_1$ & $k_2$ & $l_0$ & $l_1$ & $l_2$ & Range [MeV] \\")
    latex_table.append(r"\midrule")
    
    # Process each group for constant samples only
    for group_name, fit_params in sorted(all_fit_params.items()):
        if not fit_params:
            continue
            
        first_in_group = True
        for sample_name in sorted(fit_params.keys()):
            # Only include constant samples
            param_type = classify_parametrization_type(fit_params[sample_name])
            if param_type != "Constant":
                continue
                
            # Format the sample name for LaTeX
            latex_sample_name = latex_format_sample_name(sample_name)
            
            # Extract constant parameters (only 'a' parameters, 'b' should be zero)
            energy_const_params, angular_const_params = extract_constant_parameters(fit_params[sample_name])
            
            # Get energy range
            energy_range = get_energy_range(fit_params[sample_name])
            
            # Add row to table
            if first_in_group:
                row = f"{group_name} & {latex_sample_name} & "
                first_in_group = False
            else:
                row = f" & {latex_sample_name} & "
            
            # Add 6 parameters + energy range
            row += f"{energy_const_params[0]} & {energy_const_params[1]} & {energy_const_params[2]} & "
            row += f"{angular_const_params[0]} & {angular_const_params[1]} & {angular_const_params[2]} & "
            row += f"{energy_range} \\\\"
            
            latex_table.append(row)
        
        # Add a midrule between groups if any samples were added
        if not first_in_group:
            latex_table.append(r"\midrule")
    
    # Table footer
    latex_table.append(r"\bottomrule")
    latex_table.append(r"\end{tabular}")
    latex_table.append(r"\begin{tablenotes}")
    latex_table.append(r"\small")
    latex_table.append(r"\item \textbf{Zenith dependence:} $a_E = c_0 + c_1 \cos\theta_z + c_2 \cos^2\theta_z$ (analogously for $b_E$, $a_A$, $b_A$)")
    latex_table.append(r"\item \textbf{Note:} These resolutions are independent of neutrino energy")
    latex_table.append(r"\item \textbf{Range:} Energy range of bins used for fitting each sample")
    latex_table.append(r"\end{tablenotes}")
    latex_table.append(r"\end{table*}")
    
    return '\n'.join(latex_table)

def extract_parameters(sample_params):
    """Extract power-law parameters for a sample.
    Multiplies all coefficients by 1000 to make numbers shorter."""
    energy_a_params = ["--", "--", "--"]
    energy_b_params = ["--", "--", "--"]
    angular_a_params = ["--", "--", "--"]
    angular_b_params = ["--", "--", "--"]
    
    if 'energy_only_params' in sample_params:
        # Energy-only parametrization
        e_params = sample_params['energy_only_params']['energy']
        a_params = sample_params['energy_only_params']['angular']
        
        energy_a_params = [f"{e_params['a']*10:.2f}", "0.00", "0.00"]
        energy_b_params = [f"{e_params['b']*10:.2f}", "0.00", "0.00"]
        angular_a_params = [f"{a_params['a']*10:.2f}", "0.00", "0.00"]
        angular_b_params = [f"{a_params['b']*10:.2f}", "0.00", "0.00"]
        
    elif 'energy_res_params' in sample_params and 'angular_res_params' in sample_params:
        # Full parametrization
        e_res = sample_params['energy_res_params']
        a_res = sample_params['angular_res_params']
        
        # Energy resolution coefficients (multiply by 1000)
        energy_a_coeffs = (e_res['a'] + [0, 0, 0])[:3]
        energy_b_coeffs = (e_res['b'] + [0, 0, 0])[:3]
        energy_a_params = [f"{x*10:.2f}" for x in energy_a_coeffs]
        energy_b_params = [f"{x*10:.2f}" for x in energy_b_coeffs]
        
        # Angular resolution coefficients (multiply by 1000)
        angular_a_coeffs = (a_res['a'] + [0, 0, 0])[:3]
        angular_b_coeffs = (a_res['b'] + [0, 0, 0])[:3]
        angular_a_params = [f"{x*10:.2f}" for x in angular_a_coeffs]
        angular_b_params = [f"{x*10:.2f}" for x in angular_b_coeffs]
    
    return energy_a_params, energy_b_params, angular_a_params, angular_b_params

def extract_constant_parameters(sample_params):
    """Extract constant parameters (only 'a' coefficients since 'b' = 0)."""
    energy_const_params = ["--", "--", "--"]
    angular_const_params = ["--", "--", "--"]
    
    if 'energy_res_params' in sample_params and 'angular_res_params' in sample_params:
        e_res = sample_params['energy_res_params']
        a_res = sample_params['angular_res_params']
        
        # Only use 'a' coefficients since 'b' coefficients should be zero for constant fits
        energy_a_coeffs = (e_res['a'] + [0, 0, 0])[:3]
        energy_const_params = [f"{x:.4f}" for x in energy_a_coeffs]
        
        angular_a_coeffs = (a_res['a'] + [0, 0, 0])[:3]
        angular_const_params = [f"{x:.4f}" for x in angular_a_coeffs]
    
    return energy_const_params, angular_const_params

def get_energy_range(sample_params):
    """
    Extract energy range for a sample from stored information.
    
    Parameters:
    -----------
    sample_params : dict
        Parameters for a single sample containing energy_range info
        
    Returns:
    --------
    str
        Formatted energy range string like "[100, 5000]"
    """
    if 'energy_range' in sample_params:
        energy_range = sample_params['energy_range']
        min_energy = energy_range['min']
        max_energy = energy_range['max']
        return f"[{min_energy:.0f}, {max_energy:.0f}]"
    else:
        return "[Unknown]"

def classify_parametrization_type(sample_params):
    """
    FIXED classification function that properly handles all fit types.
    """
    # Check for energy-only parametrization first
    if 'energy_only_params' in sample_params:
        return "Energy-only"
    
    # Check for full zenith-energy parametrization
    if 'energy_res' not in sample_params and 'angular_res' not in sample_params:
        return "Insufficient"
    
    # Analyze individual zenith bin fits to determine the predominant type
    fit_types = []
    
    # Collect fit types from energy resolution
    if 'energy_res' in sample_params:
        for z_bin, params in sample_params['energy_res'].items():
            fit_type = params.get('fit_type', 'unknown')
            fit_types.append(fit_type)
    
    # If no energy resolution data, check angular resolution
    if not fit_types and 'angular_res' in sample_params:
        for z_bin, params in sample_params['angular_res'].items():
            fit_type = params.get('fit_type', 'unknown')
            fit_types.append(fit_type)
    
    if not fit_types:
        return "Insufficient"
    
    # Count occurrences of each fit type
    from collections import Counter
    type_counts = Counter(fit_types)
    most_common_type = type_counts.most_common(1)[0][0]
    
    # Map to user-friendly names - FIXED MAPPING
    type_mapping = {
        'power_law': 'Power-law',
        'linear': 'Linear',        # Make sure this matches what's stored
        'constant': 'Constant',
        'unknown': 'Unknown'
    }
    
    return type_mapping.get(most_common_type, most_common_type)

# Main generation function
def generate_all_resolution_tables(all_fit_params):
    """Generate all three resolution tables."""
    
    power_law_table = create_power_law_table(all_fit_params)
    linear_table = create_linear_table(all_fit_params)
    constant_table = create_constant_table(all_fit_params)
    
    return {
        'power_law': power_law_table,
        'linear': linear_table,
        'constant': constant_table
    }
    
def latex_format_sample_name(sample_name):
    # Define patterns and their replacements
    # Use raw strings for the replacements to avoid escape issues
    patterns = [
        (r'#bar{#nu}_{#mu}', r'$\bar{\nu}_{\mu}$'),
        (r'#bar{#nu}_{e}', r'$\bar{\nu}_{e}$'),
        (r'#nu_{#mu}', r'$\nu_{\mu}$'),
        (r'#nu_{e}', r'$\nu_{e}$'),
        (r'#pi\^{0}', r'$\pi^{0}$'),
        (r'#mu', r'$\mu$')
    ]
    
    result = sample_name
    for pattern, replacement in patterns:
        # Use string replace instead of regex to avoid escape sequence issues
        result = result.replace(pattern, replacement)
    
    return result

def fit_polynomial_flexible(x_data, y_data, max_order=2):
    """
    Fit polynomial with flexible order based on available data points.
    
    Parameters:
    -----------
    x_data : array-like
        Independent variable data
    y_data : array-like  
        Dependent variable data
    max_order : int
        Maximum polynomial order to attempt
        
    Returns:
    --------
    tuple : (coefficients, order_used, fit_type)
        coefficients: polynomial coefficients
        order_used: actual order used
        fit_type: description of fit type
    """
    n_points = len(x_data)
    
    if n_points == 0:
        return None, 0, "no_data"
    elif n_points == 1:
        # Constant fit - just use the single value
        return [y_data[0]], 0, "constant"
    elif n_points == 2:
        # Linear fit
        try:
            coeffs = np.polyfit(x_data, y_data, 1)
            return coeffs.tolist(), 1, "linear"
        except:
            # If linear fit fails, use average
            return [np.mean(y_data)], 0, "constant_fallback"
    else:
        # Try quadratic first, then linear, then constant
        for order in range(min(max_order, n_points-1), -1, -1):
            try:
                coeffs = np.polyfit(x_data, y_data, order)
                fit_type = ["constant", "linear", "quadratic"][order]
                return coeffs.tolist(), order, fit_type
            except:
                continue
        
        # Final fallback - use average
        return [np.mean(y_data)], 0, "constant_fallback"
    
def main():
    """Main function processing all sample groups with fallback strategies"""
    # Initialize AnaMaster to access all the data
    data_folder = base_dir_path+'../data/unoscillated'
    ana_master = AnaMaster(data_folder)
    
    # 1. Sub-GeV Samples
    sub_gev_samples = [
        'Sub-GeV #nu_{e}-like',
        'Sub-GeV #nu_{#mu}-like',
        'Sub-GeV #bar{#nu}_{e}-like 0 n',
        'Sub-GeV #bar{#nu}_{e}-like 1 n',
        'Sub-GeV #bar{#nu}_{#mu}-like'
    ]
    
    sub_gev_reaction_map = {
        'Sub-GeV #nu_{e}-like': 3,            # Electron sample: MCNumu
        'Sub-GeV #nu_{#mu}-like': 4,          # Muon sample: MCNueBar
        'Sub-GeV #bar{#nu}_{e}-like 0 n': 5,  # Anti-electron sample: MCNue
        'Sub-GeV #bar{#nu}_{e}-like 1 n': 5,  # Anti-electron sample: MCNue
        'Sub-GeV #bar{#nu}_{#mu}-like': 0     # Anti-muon sample: MCNC
    }
    
    sub_gev_colors = {
        'Sub-GeV #nu_{e}-like': 'blue',
        'Sub-GeV #nu_{#mu}-like': 'green', 
        'Sub-GeV #bar{#nu}_{e}-like 0 n': 'red',
        'Sub-GeV #bar{#nu}_{e}-like 1 n': 'orange',
        'Sub-GeV #bar{#nu}_{#mu}-like': 'purple'
    }
    
    print("\nProcessing Sub-GeV samples...")
    sub_gev_params = fit_and_parametrize_samples(
        ana_master, 
        "SubGeV", 
        sub_gev_samples, 
        sub_gev_reaction_map, 
        sub_gev_colors
    )
    
    # 2. Regular Multi-GeV Samples (excluding Multi-Ring)
    multi_gev_samples = [
        'Multi-GeV #nu_{e}-like',
        'Multi-GeV #bar{#nu}_{e}-like 0 n',
        'Multi-GeV #bar{#nu}_{e}-like 1 n',
        'Multi-GeV #nu_{#mu}-like',
        'Multi-GeV #bar{#nu}_{#mu}-like'
    ]
    
    # Reaction mapping for Multi-GeV
    multi_gev_reaction_map = {
        'Multi-GeV #nu_{e}-like': 3,            # Electron sample
        'Multi-GeV #bar{#nu}_{e}-like 0 n': 5,  # Anti-electron sample
        'Multi-GeV #bar{#nu}_{e}-like 1 n': 5,  # Anti-electron sample
        'Multi-GeV #nu_{#mu}-like': 4,          # Muon sample
        'Multi-GeV #bar{#nu}_{#mu}-like': 0     # Anti-muon sample
    }
    
    print("\nProcessing Multi-GeV samples...")
    multi_gev_params = fit_and_parametrize_samples(
        ana_master, 
        "MultiGeV", 
        multi_gev_samples, 
        multi_gev_reaction_map
    )
    
    # 3. Multi-Ring Multi-GeV samples (separate group)
    multi_ring_samples = [
        'Multi-GeV Multi-Ring #nu_{e}-like',
        'Multi-GeV Multi-Ring #bar{#nu}_{e}-like',
        'Multi-GeV Multi-Ring #mu-like',
        #'Multi-GeV Multi-Ring  Other'
    ]
    
    multi_ring_reaction_map = {
        'Multi-GeV Multi-Ring #nu_{e}-like': 3,
        'Multi-GeV Multi-Ring #bar{#nu}_{e}-like': 5,
        'Multi-GeV Multi-Ring #mu-like': 4,
        #'Multi-GeV Multi-Ring  Other': 4
    }
    
    print("\nProcessing Multi-Ring samples...")
    multi_ring_params = fit_and_parametrize_samples(
        ana_master, 
        "MultiRing", 
        multi_ring_samples, 
        multi_ring_reaction_map
    )
    
    # 4. Decay electron samples
    de_samples = [
        'Sub-GeV #mu-like 0 d.e.',
        'Sub-GeV #mu-like 1 d.e.',
        'Sub-GeV #mu-like 2 d.e.',
        'Sub-GeV  e-like 0 d.e.',
        'Sub-GeV  e-like 1 d.e.'
    ]
    
    # Reaction mapping for decay electron samples
    de_reaction_map = {
        'Sub-GeV #mu-like 0 d.e.': 4,
        'Sub-GeV #mu-like 1 d.e.': 4,
        'Sub-GeV #mu-like 2 d.e.': 4,
        'Sub-GeV  e-like 0 d.e.': 3,
        'Sub-GeV  e-like 1 d.e.': 3
    }
    
    print("\nProcessing decay electron samples...")
    de_params = fit_and_parametrize_samples(
        ana_master, 
        "DecayElectron", 
        de_samples, 
        de_reaction_map
    )
    
    # 5. PC and Up-μ samples (kept together)
    pc_upmu_samples = [
        'PC  Stopping',
        'PC  Through-going',
        # 'Up-#mu Stopping',
        # 'Up-#mu Non-Showering',
        # 'Up-#mu Showering'
    ]
    
    # Use reaction 0 for all PC and Up-μ samples as requested
    pc_upmu_reaction_map = {s: 4 for s in pc_upmu_samples}
    
    # Define distinct colors for each sample to make them easily distinguishable
    pc_upmu_colors = {
        'PC  Stopping': 'blue',
        'PC  Through-going': 'green',
        'Up-#mu Stopping': 'red',
        'Up-#mu Non-Showering': 'purple',
        'Up-#mu Showering': 'orange'
    }
    
    print("\nProcessing PC and Up-μ samples...")
    pc_upmu_params = fit_and_parametrize_samples(
        ana_master, 
        "Other", 
        pc_upmu_samples, 
        pc_upmu_reaction_map,
        pc_upmu_colors
    )
    
    return {
        'SubGeV': sub_gev_params,
        'MultiGeV': multi_gev_params,
        'MultiRing': multi_ring_params,
        'DecayElectron': de_params,
        'Other': pc_upmu_params
    }

In [7]:
# Run the main function
if __name__ == "__main__":
    all_fit_params = main()
    
    tables = generate_all_resolution_tables(all_fit_params)

    output_dir = create_output_dir()
    tab = generate_all_resolution_tables(all_fit_params)
    table_filename = os.path.join(output_dir, 'resolution_params_power_law.tex')
    with open(table_filename, 'w') as f:
        f.write(tab['power_law'])

    table_filename = os.path.join(output_dir, 'resolution_params_linear.tex')
    with open(table_filename, 'w') as f:
        f.write(tab['linear'])

MASK BINS:  True

Processing Sub-GeV samples...
Processing: Sub-GeV #nu_{e}-like
Processing: Sub-GeV #bar{#nu}_{e}-like 0 n
Processing: Sub-GeV #bar{#nu}_{e}-like 1 n
Processing: Sub-GeV #nu_{#mu}-like
Processing: Sub-GeV #bar{#nu}_{#mu}-like

Analyzing Sub-GeV #bar{#nu}_{#mu}-like:
  Number of zenith bins: 10
    Zenith bin -0.2_0.0: 5 points
      Energy resolution: Power law fit successful (a=0.2650, b=0.4301)
      Angular resolution: Power law fit successful (a=0.3510, b=0.4362)
    Zenith bin -0.5_-0.2: 5 points
      Energy resolution: Power law fit successful (a=0.2545, b=0.4757)
      Angular resolution: Power law fit successful (a=0.3447, b=0.4002)
    Zenith bin -0.6_-0.5: 5 points
      Energy resolution: Power law fit successful (a=0.2518, b=0.5119)
      Angular resolution: Power law fit successful (a=0.3213, b=0.4443)
    Zenith bin -0.8_-0.6: 5 points
      Energy resolution: Power law fit successful (a=0.2616, b=0.4365)
      Angular resolution: Power law fit successfu